In [1]:
from torchvision.models import resnet50, resnet18
from torchvision.models import ResNet50_Weights
from torch import nn 
import torch 

class Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = resnet50(weights=ResNet50_Weights)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        
    def features(self, x) -> torch.Tensor:
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)

        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)

        x = self.model.avgpool(x)
        x = torch.flatten(x, 1)
        return x

    def forward(self, x):
        return self.model.forward(x)
    

In [2]:
from torch.utils.data import Dataset, DataLoader
from os.path import join, exists
from os import listdir
import pandas as pd 
from PIL import Image
from torch.optim import SGD
import matplotlib.pyplot as plt 
from awa import AwA
from torchvision.models import resnet50, resnet18
from torchvision.models import ResNet50_Weights
from pytorch_ood.utils import ToUnknown, ToRGB
from torchvision.transforms import ToTensor, Compose, Resize 
from torchvision import transforms
from pytorch_ood.utils import ToRGB
import torch 
from pytorch_ood.dataset.img import Textures, FractalDataset, FoolingImages, ImageNetA, ImageNetO, ImageNetR
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset

transform = transforms.Compose([            
     ToRGB(),
     transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize(
     mean=[0.485, 0.456, 0.406],
     std=[0.229, 0.224, 0.225]
     )
])

device = "cuda:0"

model = Model(num_classes=50) # weights=ResNet50_Weights
model.eval().to(device)


def get_loader_for_name(name, root, subset):
    if name == "awa":
        dataset = AwA(root=root, transform=transform)
        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [30000, 7322], generator=torch.Generator().manual_seed(123))
        if subset == "test":
            dataset = test_dataset
        elif subset == "train":
            dataset = train_dataset
        else:
            raise ValueError
    else:
        names = {
            "textures": Textures,
            "fractals": FractalDataset,
            "fooling": FoolingImages,
            "imagenet-a": ImageNetA,
            "imagenet-o": ImageNetO,
            "imagenet-r": ImageNetR
        }
        clazz = names[name]
        dataset = clazz(root=root, transform=transform, target_transform=ToUnknown(), download=True)
        
    loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False, num_workers=10)
    return loader     
        
        
def load_dataset_features(name, root, subset="test", full=True):
    print(f"Loading {name}-{subset}")
    
    path = join("data", f"features-{name}-{subset}.pt")
    
    if exists(path):
        if name == "awa":
            x, y, z = torch.load(path, map_location="cpu")
            if full:
                return TensorDataset(x, y, z)
            else:
                return TensorDataset(x, y)
        else:
            x, y = torch.load(path, map_location="cpu")
            return TensorDataset(x, y)
    else:
        loader = get_loader_for_name(name, root, subset)

        bar =  tqdm(enumerate(loader), total=len(loader))
        features = []
        ys = []
        zs = []

        with torch.no_grad():
            for i, batch in bar:
                if name == "awa":
                    x, y, z  = batch
                    x = x.to(device)
                    outputs = model.features(x)
                    features.append(outputs)
                    ys.append(y)
                    zs.append(z)
                else:
                    x, y  = batch
                    x = x.to(device)
                    outputs = model.features(x)
                    features.append(outputs)
                    ys.append(y)

        xs = torch.cat(features)
        ys = torch.cat(ys)
        if name == "awa": 
            zs = torch.cat(zs)
        
        if name == "awa": 
            torch.save((xs, ys, zs), path)
            if full:
                return TensorDataset(xs.cpu(), ys.cpu(), zs.cpu())
            else:
                return TensorDataset(xs.cpu(), ys.cpu())
        else:
            torch.save((xs, ys), path)
            return TensorDataset(xs.cpu(), ys.cpu())
        

/home/ki/.local/share/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
root = "/home/ki/datasets/" 

In [4]:
train_ds = load_dataset_features("awa", root, subset="train")
test_ds = load_dataset_features("awa", root, subset="test")

Loading awa-train


  0%|          | 0/469 [00:00<?, ?it/s]

Loading awa-test


  0%|          | 0/115 [00:00<?, ?it/s]

In [5]:
mini_model = nn.Linear(2048, 50)

learning_rate = 1e-3
momentum = 0.9
criterion = nn.CrossEntropyLoss()
optimizer = SGD(mini_model.parameters(), lr=learning_rate, momentum=momentum, nesterov=True)

_ = mini_model.to(device)

train_ld = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=10)
test_ld = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=True, num_workers=4)


for epoch in range(3):
    running_loss = 0.0
    bar =  tqdm(enumerate(train_ld), total=len(train_ld))
    for i, batch in bar:
        x, y, z  = batch
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = mini_model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss = 0.8* running_loss + 0.2 * loss.item()
        bar.set_postfix({"loss": running_loss})

    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_ld:
            x, y, z  = batch
            x, y = x.to(device), y.to(device)

            outputs = mini_model(x)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    
    bar.set_postfix({"loss": running_loss, "acc": correct / total})
    print(f'Accuracy on test images: {correct / total:.2%}')
    
mini_model.eval()

  0%|          | 0/469 [00:00<?, ?it/s]

Accuracy on test images: 87.56%


  0%|          | 0/469 [00:00<?, ?it/s]

Accuracy on test images: 90.00%


  0%|          | 0/469 [00:00<?, ?it/s]

Accuracy on test images: 90.85%


In [69]:
# train with different attributes 
def train_for_attribute(att_index):
    train_ds = load_dataset_features("awa", root, subset="train")
    test_ds = load_dataset_features("awa", root, subset="test")
    mini_model = nn.Linear(2048, 2)

    learning_rate = 1e-3
    momentum = 0.9
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(mini_model.parameters(), lr=learning_rate, momentum=momentum, nesterov=True)

    _ = mini_model.to(device)

    train_ld = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=10)
    test_ld = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=True, num_workers=4)

    for epoch in range(3):
        running_loss = 0.0
        for batch in train_ld:
            x, y, z  = batch
            
            x, z = x.to(device), z.to(device)
            z = z[:,att_index].long()
            
            optimizer.zero_grad()
            outputs = mini_model(x)
            loss = criterion(outputs, z)
            loss.backward()
            optimizer.step()

            running_loss = 0.8* running_loss + 0.2 * loss.item()

        correct = 0
        total = 0

        with torch.no_grad():
            for batch in test_ld:
                x, y, z  = batch
                x, z = x.to(device), z.to(device)
                z = z[:,att_index].long()
            
                outputs = mini_model(x)
                _, predicted = torch.max(outputs.data, 1)
                total += z.size(0)
                correct += (predicted == z).sum().item()

        bar.set_postfix({"loss": running_loss, "acc": correct / total})
        print(f'Accuracy on test images: {correct / total:.2%}')
    
    return mini_model 


In [70]:
models = []

awads = AwA(root=root)
    
for i in range(85):
    print(f"Training model for {i} -> {awads.att_idx_to_name[i]}")
    mini_model_i = train_for_attribute(i)
    model.eval()
    models.append(mini_model_i)

Training model for 0 -> black
Loading awa-train
Loading awa-test
Accuracy on test images: 88.31%
Accuracy on test images: 89.61%
Accuracy on test images: 89.87%
Training model for 1 -> white
Loading awa-train
Loading awa-test
Accuracy on test images: 89.66%
Accuracy on test images: 90.96%
Accuracy on test images: 91.19%
Training model for 2 -> blue
Loading awa-train
Loading awa-test


KeyboardInterrupt: 

In [8]:
from pytorch_ood.api import Detector
from typing import List 

class EnsembleDetector(Detector):
    """
    Ensemble of several OOD detectors 
    """
    
    def __init__(self, detectors: List[Detector]):
        self.detectors = detectors
    
    def predict(self, x):
        with torch.no_grad():
            scores = [d(x) for d in self.detectors]
            # scores = [l.softmax(dim=1).max(dim=1).values for l in scores]
            scores = torch.stack(scores, dim=1).mean(dim=1)
            # scores = torch.stack(scores, dim=1).mean(dim=1)
        return scores
    
    def fit():
        pass 
    

In [9]:
from pytorch_ood.dataset.img import Textures, FractalDataset, FoolingImages, ImageNetA, ImageNetO, ImageNetR

In [10]:
import pytorch_ood
from pytorch_ood.utils import ToRGB
from pytorch_ood.detector import MaxSoftmax, EnergyBased, Mahalanobis
from pytorch_ood.utils import OODMetrics, ToRGB, ToUnknown
from pandas import DataFrame


def ood_label(x):
    if x == True:
        return "Normal"
    else:
        return "Anomaly"


def eval_on_loader(detector, loader):
    scores = []
    ys = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            scores.append(detector(x).cpu())
            ys.append(y)

    scores = torch.cat(scores)
    ys = torch.cat(ys)
    return scores, ys

    
def evaluate(detector):
    test_ds = load_dataset_features("awa", root, subset="test", full=False)
    
    metrics = []
    
    for dataset in ["textures", "fractals", "fooling", "imagenet-a", "imagenet-o", "imagenet-r"]:
        print(f"--> {dataset}")
        dataset_out_test = load_dataset_features(dataset, root, subset="test")
        loader = DataLoader(dataset_out_test +  test_ds, batch_size=16, shuffle=False, num_workers=10)

        scores, ys = eval_on_loader(detector, loader)

        m = OODMetrics()
        m.update(scores, ys)

        met = m.compute()
        met["Dataset"] = dataset
        metrics.append(met)
    
    return metrics 

In [11]:
results = []
seed_rep = 0

from pytorch_ood.detector import MaxSoftmax, EnergyBased

ensemble_detectors = [MaxSoftmax(m) for m in models]

detectors = {
    "MaxSoftmax": MaxSoftmax(mini_model),
    "EnergyBased": EnergyBased(mini_model), 
    "Ensemble": EnsembleDetector(ensemble_detectors + [MaxSoftmax(mini_model)])
}

for name, detector in detectors.items():
    print(f"-> {name}")
    metrics = evaluate(detector)
    for m in metrics:
        m.update({
            "Method": name,
            "Rep": seed_rep
        })
    results += metrics


result_df = pd.DataFrame(results)

-> MaxSoftmax
Loading awa-test
--> textures
Loading textures-test


  0%|          | 0/89 [00:00<?, ?it/s]

--> fractals
Loading fractals-test


  0%|          | 0/223 [00:00<?, ?it/s]

/home/ki/.local/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


...

  0%|          | 0/32 [00:00<?, ?it/s]

/home/ki/.local/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 4. Skipping tag 37381
  warnings.warn(
/home/ki/.local/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 37386
  warnings.warn(


--> imagenet-r
Loading imagenet-r-test


  0%|          | 0/469 [00:00<?, ?it/s]

-> EnergyBased
Loading awa-test
--> textures
Loading textures-test
--> fractals
Loading fractals-test
--> fooling
Loading fooling-test
--> imagenet-a
Loading imagenet-a-test
--> imagenet-o
Loading imagenet-o-test
--> imagenet-r
Loading imagenet-r-test
-> Ensemble
Loading awa-test
--> textures
Loading textures-test
--> fractals
Loading fractals-test
--> fooling
Loading fooling-test
--> imagenet-a
Loading imagenet-a-test
--> imagenet-o
Loading imagenet-o-test
--> imagenet-r
Loading imagenet-r-test


In [12]:
print((result_df.groupby(by=["Method"]).agg(["mean"]) * 100)[["AUROC", "AUPR-IN", "AUPR-OUT", "FPR95TPR"]].to_latex(float_format="%.2f"))

\begin{tabular}{lrrrr}
\toprule
{} & AUROC & AUPR-IN & AUPR-OUT & FPR95TPR \\
{} &  mean &    mean &     mean &     mean \\
Method      &       &         &          &          \\
\midrule
EnergyBased & 98.68 &   98.35 &    98.46 &     5.61 \\
Ensemble    & 92.21 &   86.56 &    93.14 &    21.12 \\
MaxSoftmax  & 97.70 &   97.05 &    97.32 &    10.06 \\
\bottomrule
\end{tabular}



In [13]:
result_df

,AUROC,AUPR-IN,AUPR-OUT,ACC95TPR,FPR95TPR,Dataset,Method,Rep
0,0.969938,0.958632,0.976465,0.903564,0.132204,textures,MaxSoftmax,0
1,0.978644,0.987127,0.966914,0.935744,0.092051,fractals,MaxSoftmax,0
2,0.971794,0.976565,0.964718,0.918774,0.123873,fooling,MaxSoftmax,0
3,0.980464,0.978153,0.982724,0.933770,0.082901,imagenet-a,MaxSoftmax,0
4,0.978154,0.927726,0.993704,0.912787,0.097378,imagenet-o,MaxSoftmax,0
5,0.982902,0.995073,0.954836,0.945073,0.075116,imagenet-r,MaxSoftmax,0
6,0.980357,0.973539,0.984403,0.931415,0.082901,textures,EnergyBased,0
7,0.988655,0.993171,0.982605,0.951043,0.046982,fractals,EnergyBased,0
8,0.981863,0.985344,0.976183,0.938575,0.077028,fooling,EnergyBased,0
9,0.989205,0.988422,0.990036,0.950801,0.048484,imagenet-a,EnergyBased,0


In [66]:
class SemanticDetector(Detector):
    """
    
    """ 
    
    def __init__(self, label_model, att_models: List[nn.Module], target_vectors, n=84):
        """
        Models and target vectors 
        """
        self.label_model = label_model 
        self.att_model = att_models 
        self.target_vectors = target_vectors 
        self.n = n
    
    def predict(self, x):
        values, labels = self.label_model(x).softmax(dim=1).max(dim=1)
        outputs = [m(x).max(dim=1).indices for m in models]
        att_vectors = torch.stack(outputs, dim=1).cpu()
        
        sat = []
        
        for l, att_vector in zip(labels, att_vectors):
            s = att_vector == target_vectors[l]
            sat.append(s.sum() >= self.n)
        
        sat = torch.tensor(sat).float()
        # print(sat.sum())
        return - values.cpu() * sat
    
    def fit():
        pass 

In [40]:
results = []
seed_rep = 0

from pytorch_ood.detector import MaxSoftmax, EnergyBased

detectors = [MaxSoftmax(m) for m in models] + [MaxSoftmax(mini_model)]

for n, detector in tqdm(enumerate(detectors), total=len(detectors)):
    print(f"-> {n}")
    metrics = evaluate(detector)
    for m in metrics:
        m.update({
            "Detector": n,
            "Rep": seed_rep
        })
    results += metrics

result_df = pd.DataFrame(results)

  0%|          | 0/86 [00:00<?, ?it/s]

-> 0
Loading awa-test
--> textures
Loading textures-test
--> fractals
Loading fractals-test
--> fooling
Loading fooling-test


KeyboardInterrupt: 

In [17]:
print((result_df.groupby(by=["Detector"]).agg(["mean"]) * 100)[["AUROC", "AUPR-IN", "AUPR-OUT", "FPR95TPR"]].to_latex(float_format="%.2f"))

\begin{tabular}{lrrrr}
\toprule
{} & AUROC & AUPR-IN & AUPR-OUT & FPR95TPR \\
{} &  mean &    mean &     mean &     mean \\
Detector &       &         &          &          \\
\midrule
0        & 70.22 &   66.92 &    65.93 &    78.82 \\
1        & 72.76 &   68.87 &    68.64 &    74.81 \\
2        & 58.68 &   56.50 &    53.90 &    89.83 \\
3        & 72.14 &   68.68 &    67.23 &    77.16 \\
4        & 75.61 &   71.54 &    71.89 &    70.00 \\
5        & 55.06 &   54.89 &    51.32 &    91.83 \\
6        & 49.52 &   51.54 &    47.60 &    93.46 \\
7        & 57.28 &   57.92 &    52.70 &    91.08 \\
8        & 68.40 &   65.54 &    63.21 &    80.91 \\
9        & 60.01 &   59.50 &    55.57 &    88.69 \\
10       & 54.31 &   56.73 &    49.33 &    93.85 \\
11       & 67.18 &   64.40 &    61.80 &    83.25 \\
12       & 69.88 &   66.63 &    65.10 &    79.30 \\
13       & 80.73 &   75.68 &    78.53 &    58.54 \\
14       & 75.07 &   72.34 &    69.35 &    76.16 \\
15       & 76.56 &   72.12 &    74.

In [27]:
target_vectors = torch.tensor(awads.predicates.values).long()
target_vectors

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 1, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        ...,
        [1, 1, 0,  ..., 1, 1, 0],
        [1, 1, 0,  ..., 0, 0, 1],
        [0, 1, 1,  ..., 0, 0, 1]])

In [63]:
detector = SemanticDetector(mini_model, models, target_vectors=target_vectors)

In [64]:
detector(x) == 0.0

tensor(8.)


tensor([False,  True, False,  True,  True,  True,  True, False,  True,  True,
        False, False, False,  True,  True,  True,  True, False,  True,  True,
         True, False,  True,  True,  True,  True])

In [67]:
results = []
seed_rep = 0

from pytorch_ood.detector import MaxSoftmax, EnergyBased

ensemble_detectors = [MaxSoftmax(m) for m in models]

detectors = {
    "Semantic": SemanticDetector(mini_model, models, target_vectors=target_vectors)
}

for name, detector in detectors.items():
    print(f"-> {name}")
    metrics = evaluate(detector)
    for m in metrics:
        m.update({
            "Method": name,
            "Rep": seed_rep
        })
    results += metrics


result_df = pd.DataFrame(results)

-> Semantic
Loading awa-test
--> textures
Loading textures-test
--> fractals
Loading fractals-test
--> fooling
Loading fooling-test
--> imagenet-a
Loading imagenet-a-test
--> imagenet-o
Loading imagenet-o-test
--> imagenet-r
Loading imagenet-r-test


In [68]:
result_df

,AUROC,AUPR-IN,AUPR-OUT,ACC95TPR,FPR95TPR,Dataset,Method,Rep
0,0.747917,0.801264,0.889107,0.715245,0.502868,textures,Semantic,0
1,0.748114,0.896711,0.831838,0.828419,0.502868,fractals,Semantic,0
2,0.747591,0.864790,0.850182,0.786168,0.502868,fooling,Semantic,0
3,0.748114,0.834103,0.872030,0.750422,0.502868,imagenet-a,Semantic,0
4,0.747831,0.674827,0.945548,0.604591,0.502868,imagenet-o,Semantic,0
5,0.748203,0.945204,0.794732,0.900354,0.502868,imagenet-r,Semantic,0
